In [4]:
import cv2
import mediapipe as mp
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

# LOAD MLP (string class labels)
with open("mlp_model.pkl", "rb") as f:
    mlp_model = pickle.load(f)

print(" Loaded MLP model.")

feature_extractor = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    pooling="avg",
    input_shape=(224, 224, 3)
)

print(" EfficientNet loaded.")
IMG_SIZE = 224

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)


# HEATMAP + SKELETON

def draw_gaussian(heatmap, x, y, sigma=12):
    xx, yy = np.meshgrid(np.arange(256), np.arange(256))
    return heatmap + np.exp(-((xx - x)**2 + (yy - y)**2) / (2*sigma**2))

def generate_heatmap_and_keypoints(frame):
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    heatmap = np.zeros((256, 256), dtype=np.float32)
    keypoints = []

    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            x = int(lm.x * 256)
            y = int(lm.y * 256)
            keypoints.append((x, y))
            heatmap = draw_gaussian(heatmap, x, y)

        for a, b in mp_pose.POSE_CONNECTIONS:
            if a < len(keypoints) and b < len(keypoints):
                cv2.line(heatmap, keypoints[a], keypoints[b], 1.0, 2)

    if heatmap.max() == 0:
        return None, None

    heatmap = (heatmap / heatmap.max() * 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    return heatmap, keypoints

def extract_features(img):
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    feat = feature_extractor.predict(img, verbose=0)
    return feat.flatten()


# FEEDBACK SYSTEM
def feedback_for_pose(pose, keypoints, confidence):
    if confidence < 0.55:
        return "Pose unclear. Try standing straight."

    # Sitting detection
    if keypoints:
        hip_y = keypoints[23][1]
        shoulder_y = keypoints[11][1]

        if abs(hip_y - shoulder_y) < 40:  
            return "You seem to be sitting. Stand up for yoga pose."

    # Custom feedback per yoga pose
    if pose == "Goddess":
        return "Open your hips wider and keep your spine upright."

    if pose == "Tree":
        return "Balance on one leg and keep hands steady."

    if pose == "Plank":
        return "Keep your body in a straight line."

    if pose == "Side Plank":
        return "Raise your upper arm straight above you."

    if pose == "Warrior":
        return "Extend your arms and bend front knee properly."

    if pose == "Downdog":
        return "Push hips upward and straighten your back."

    return "Good posture!"


# REAL TIME LOOP
cap = cv2.VideoCapture(0)

print("🎥 Webcam started. Press Q to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    heatmap, keypoints = generate_heatmap_and_keypoints(frame)

    if heatmap is None:
        cv2.putText(frame, "No person detected", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        cv2.imshow("Yoga Pose Classification", frame)
        if cv2.waitKey(1) == ord('q'):
            break
        continue

    # Extract features
    features = extract_features(heatmap)

    # Prediction + Confidence
    proba = mlp_model.predict_proba([features])[0]
    confidence = float(np.max(proba))
    pred_label = mlp_model.classes_[np.argmax(proba)]

    # Feedback system
    fb = feedback_for_pose(pred_label, keypoints, confidence)

    # Display results
    cv2.putText(frame, f"Pose: {pred_label} ({confidence*100:.1f}%)",
                (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 3)
    cv2.putText(frame, f"Feedback: {fb}",
                (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,200,255), 2)

    cv2.imshow("Yoga Pose Classification", frame)
    cv2.imshow("Heatmap", heatmap)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


 Loaded MLP model.
 EfficientNet loaded.
🎥 Webcam started. Press Q to quit
